In [ ]:
import os, cv2
import mmcv
import torch
from tqdm import tqdm
from argparse import ArgumentParser
from mmseg.apis import init_model, inference_model

import pandas as pd
import numpy as np
import json
import numpy as np
import matplotlib.pyplot as plt

palette = np.array([[0, 0, 0], [0, 0, 255]])
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def check():
    check_dir = "/home/jovyan/work/prj_data/open/train_img"
    img_list = [f for f in os.listdir(check_dir) if os.path.splitext(f)[1] == ".png"]
    img_list.sort()
    anno_pd = pd.read_csv("/home/jovyan/work/prj_data/open/train.csv")
    save_dir = "/home/jovyan/work/work_space/data EDA/check_outlier"
    alpha = 0.65
    for img_name in tqdm(img_list):
        img_path = os.path.join(check_dir, img_name)
        img = cv2.imread(img_path)
        mask_rle = anno_pd[anno_pd["img_id"]==os.path.splitext(img_name)[0]]["mask_rle"].values.tolist()[0]
        mask = rle_decode(mask_rle, (img.shape[0], img.shape[1]))
        mask = palette[mask]
        blended = cv2.addWeighted(img, alpha, mask, (1-alpha), 0, dtype = cv2.CV_8U)
        cv2.imwrite(os.path.join(save_dir, img_name), blended)
        
check()